(emcee_conversion)=
# Converting emcee objects to DataTree

{class}`~datatree.DataTree` is the data format ArviZ relies on.

This page covers multiple ways to generate a `DataTree` from emcee objects.

```{seealso}

- Conversion from Python, numpy or pandas objects
- {ref}`xarray_for_arviz` for an overview of `InferenceData` and its role within ArviZ.
- {ref}`schema` describes the structure of `InferenceData` objects and the assumptions made by ArviZ to ease your exploratory analysis of Bayesian models.
```

We will start by importing the required packages and defining the model. The famous 8 school model.

In [1]:
import arviz_base as az
import numpy as np
import emcee

In [2]:
J = 8
y_obs = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])

In [3]:
def log_prior_8school(theta):
    mu, tau, eta = theta[0], theta[1], theta[2:]
    # Half-cauchy prior, hwhm=25
    if tau < 0:
        return -np.inf
    prior_tau = -np.log(tau**2 + 25**2)
    prior_mu = -((mu / 10) ** 2)  # normal prior, loc=0, scale=10
    prior_eta = -np.sum(eta**2)  # normal prior, loc=0, scale=1
    return prior_mu + prior_tau + prior_eta


def log_likelihood_8school(theta, y, s):
    mu, tau, eta = theta[0], theta[1], theta[2:]
    return -(((mu + tau * eta - y) / s) ** 2)


def lnprob_8school(theta, y, s):
    prior = log_prior_8school(theta)
    like_vect = log_likelihood_8school(theta, y, s)
    like = np.sum(like_vect)
    return like + prior

In [4]:
nwalkers = 40  # called chains in ArviZ
ndim = J + 2
draws = 1500
pos = np.random.normal(size=(nwalkers, ndim))
pos[:, 1] = np.absolute(pos[:, 1])
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob_8school, args=(y_obs, sigma))
sampler.run_mcmc(pos, draws);

## Manually set variable names
This first example will show how to convert manually setting the variable names only, leaving everything else to ArviZ defaults.

In [5]:
# define variable names, it cannot be inferred from emcee
var_names = ["mu", "tau"] + ["eta{}".format(i) for i in range(J)]
idata1 = az.from_emcee(sampler, var_names=var_names)
idata1

DataTree('None', parent=None)
├── DataTree('posterior')
│       Dimensions:  (draw: 1500, chain: 40)
│       Coordinates:
│         * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
│         * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
│       Data variables:
│           mu       (draw, chain) float64 0.04892 0.4058 -0.4955 ... 0.92 6.357 6.915
│           tau      (draw, chain) float64 2.306 0.1492 2.226 ... 24.55 15.16 0.8366
│           eta0     (draw, chain) float64 -0.6786 1.546 -0.2988 ... 0.8633 0.2857
│           eta1     (draw, chain) float64 -0.1647 0.01271 1.822 ... -0.6913 -0.3475
│           eta2     (draw, chain) float64 -0.5788 -0.6777 -0.9958 ... -0.1428 -0.1201
│           eta3     (draw, chain) float64 -0.9262 0.363 1.34 ... 0.3492 -0.1585 1.011
│           eta4     (draw, chain) float64 1.789 0.1214 -0.5256 ... -0.1618 -1.608
│           eta5     (draw, chain) float64 0.3746 0.1143 1.87 ... 0.3285 -0.2215
│           eta6     (draw, chain) float64 0.2838 -0.6526 0.2947 ... 0.6523 0.1229
│           eta7     (draw, chain) float64 -1.689 0.5856 0.5864 ... -0.9638 -0.3025
│       Attributes:
│           created_at:                 2023-06-16T00:40:42.099033
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('observed_data')
│       Dimensions:      (arg_0_dim_0: 8, arg_1_dim_0: 8)
│       Coordinates:
│         * arg_0_dim_0  (arg_0_dim_0) int64 0 1 2 3 4 5 6 7
│         * arg_1_dim_0  (arg_1_dim_0) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           arg_0        (arg_0_dim_0) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
│           arg_1        (arg_1_dim_0) float64 15.0 10.0 16.0 11.0 9.0 11.0 10.0 18.0
│       Attributes:
│           created_at:                 2023-06-16T00:40:42.092684
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
└── DataTree('sample_stats')
        Dimensions:  (draw: 1500, chain: 40)
        Coordinates:
          * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
          * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
        Data variables:
            lp       (draw, chain) float64 -23.76 -18.08 -24.63 ... -11.52 -14.46 -15.33
        Attributes:
            created_at:                 2023-06-16T00:40:42.090166
            creation_library:           ArviZ
            creation_library_version:   0.1
            creation_library_language:  Python
            inference_library:          emcee
            inference_library_version:  3.1.4

ArviZ has stored the posterior variables with the provided names as expected, but it has also included other useful information in the `InferenceData` object. The log probability of each sample is stored in the `sample_stats` group under the name `lp` and all the arguments passed to the sampler as `args` have been saved in the `observed_data` group.

It can also be useful to perform a burn in cut to the MCMC samples (see :meth:`arviz.InferenceData.sel` for more details)

In [6]:
#idata1.sel(draw=slice(100, None))

From an InferenceData object, ArviZ's native data structure, the {func}`posterior plot <arviz.plot_posterior>` of a few variables can be done in one line:

In [7]:
#az.plot_posterior(idata1, var_names=["mu", "tau", "eta4"])

## Structuring the posterior as multidimensional variables

This way of calling ``from_emcee`` stores each `eta` as a different variable, called `eta#`, 
however, they are in fact different dimensions of the same variable. 
This can be seen in the code of the likelihood and prior functions, where `theta` is unpacked as:

    mu, tau, eta = theta[0], theta[1], theta[2:]
    
ArviZ has support for multidimensional variables, and there is a way to tell it how to split the variables like it was done in the likelihood and prior functions:

In [8]:
idata2 = az.from_emcee(sampler, slices=[0, 1, slice(2, None)])
idata2

DataTree('None', parent=None)
├── DataTree('posterior')
│       Dimensions:      (draw: 1500, chain: 40, var_2_dim_0: 8)
│       Coordinates:
│         * draw         (draw) int64 0 1 2 3 4 5 6 ... 1494 1495 1496 1497 1498 1499
│         * chain        (chain) int64 0 1 2 3 4 5 6 7 8 ... 31 32 33 34 35 36 37 38 39
│         * var_2_dim_0  (var_2_dim_0) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           var_0        (draw, chain) float64 0.04892 0.4058 -0.4955 ... 6.357 6.915
│           var_1        (draw, chain) float64 2.306 0.1492 2.226 ... 24.55 15.16 0.8366
│           var_2        (draw, chain, var_2_dim_0) float64 -0.6786 -0.1647 ... -0.3025
│       Attributes:
│           created_at:                 2023-06-16T00:40:42.482732
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('observed_data')
│       Dimensions:      (arg_0_dim_0: 8, arg_1_dim_0: 8)
│       Coordinates:
│         * arg_0_dim_0  (arg_0_dim_0) int64 0 1 2 3 4 5 6 7
│         * arg_1_dim_0  (arg_1_dim_0) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           arg_0        (arg_0_dim_0) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
│           arg_1        (arg_1_dim_0) float64 15.0 10.0 16.0 11.0 9.0 11.0 10.0 18.0
│       Attributes:
│           created_at:                 2023-06-16T00:40:42.481120
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
└── DataTree('sample_stats')
        Dimensions:  (draw: 1500, chain: 40)
        Coordinates:
          * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
          * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
        Data variables:
            lp       (draw, chain) float64 -23.76 -18.08 -24.63 ... -11.52 -14.46 -15.33
        Attributes:
            created_at:                 2023-06-16T00:40:42.479558
            creation_library:           ArviZ
            creation_library_version:   0.1
            creation_library_language:  Python
            inference_library:          emcee
            inference_library_version:  3.1.4

After checking the default variable names, the trace of one dimension of eta can be plotted using ArviZ syntax:

In [9]:
#az.plot_trace(idata2, var_names=["var_2"], coords={"var_2_dim_0": 4});

## `blobs`: unlock sample stats, posterior predictive and miscellanea

Emcee does not store per-draw sample stats, however, it has a functionality called
blobs that allows to store any variable on a per-draw basis. It can be used
to store some sample_stats or even posterior_predictive data. 

You can modify the probability function to use this ``blobs`` functionality and store the pointwise log likelihood,
then rerun the sampler using the new function:

In [10]:
def lnprob_8school_blobs(theta, y, s):
    prior = log_prior_8school(theta)
    like_vect = log_likelihood_8school(theta, y, s)
    like = np.sum(like_vect)
    return like + prior, like_vect


sampler_blobs = emcee.EnsembleSampler(
    nwalkers,
    ndim,
    lnprob_8school_blobs,
    args=(y_obs, sigma),
)
sampler_blobs.run_mcmc(pos, draws);

You can now use the `blob_names` argument to indicate how to store this blob-defined variable. As the group is not specified, it will go to sample_stats.
Note that the argument blob_names is added to the arguments covered in the previous examples and we are also introducing `coords` and `dims` arguments to show the power and flexibility of the converter. For more on `coords` and `dims` see `page_in_construction`.

In [11]:
dims = {"eta": ["school"], "log_likelihood": ["school"]}
idata3 = az.from_emcee(
    sampler_blobs,
    var_names=["mu", "tau", "eta"],
    slices=[0, 1, slice(2, None)],
    blob_names=["y"],
    dims=dims,
    coords={"school": range(8)},
)
idata3

DataTree('None', parent=None)
├── DataTree('posterior')
│       Dimensions:  (draw: 1500, chain: 40, school: 8)
│       Coordinates:
│         * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
│         * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
│         * school   (school) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           mu       (draw, chain) float64 0.04892 0.4058 -0.4955 ... 0.92 6.357 6.915
│           tau      (draw, chain) float64 2.306 0.1492 2.226 ... 24.55 15.16 0.8366
│           eta      (draw, chain, school) float64 -0.6786 -0.1647 ... 0.1229 -0.3025
│       Attributes:
│           created_at:                 2023-06-16T00:40:44.547558
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('observed_data')
│       Dimensions:      (arg_0_dim_0: 8, arg_1_dim_0: 8)
│       Coordinates:
│         * arg_0_dim_0  (arg_0_dim_0) int64 0 1 2 3 4 5 6 7
│         * arg_1_dim_0  (arg_1_dim_0) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           arg_0        (arg_0_dim_0) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
│           arg_1        (arg_1_dim_0) float64 15.0 10.0 16.0 11.0 9.0 11.0 10.0 18.0
│       Attributes:
│           created_at:                 2023-06-16T00:40:44.545948
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('log_likelihood')
│       Dimensions:  (draw: 1500, chain: 40, y_dim_0: 8)
│       Coordinates:
│         * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
│         * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
│         * y_dim_0  (y_dim_0) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           y        (draw, chain, y_dim_0) float64 -3.872 -0.694 ... -1.206 -0.08795
│       Attributes:
│           created_at:                 2023-06-16T00:40:44.542903
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
└── DataTree('sample_stats')
        Dimensions:  (draw: 1500, chain: 40)
        Coordinates:
          * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
          * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
        Data variables:
            lp       (draw, chain) float64 -23.76 -18.08 -24.63 ... -11.52 -14.46 -15.33
        Attributes:
            created_at:                 2023-06-16T00:40:44.545000
            creation_library:           ArviZ
            creation_library_version:   0.1
            creation_library_language:  Python
            inference_library:          emcee
            inference_library_version:  3.1.4

## Multi-group blobs

You might even have more complicated blobs, each corresponding to a different group of the InferenceData object. Moreover, you can store the variables passed to the ``EnsembleSampler`` via the ``args`` argument in observed or constant data groups. This is shown in the example below:

In [12]:
sampler_blobs.blobs[0, 1]

array([-3.32784122e+00, -5.76423166e-01, -4.26615268e-02, -3.53483214e-01,
       -2.50328863e-02, -2.75246231e-03, -3.12989363e+00, -4.08660379e-01])

In [13]:
def lnprob_8school_blobs(theta, y, sigma):
    mu, tau, eta = theta[0], theta[1], theta[2:]
    prior = log_prior_8school(theta)
    like_vect = log_likelihood_8school(theta, y, sigma)
    like = np.sum(like_vect)
    # store pointwise log likelihood, useful for model comparison with az.loo or az.waic
    # and posterior predictive samples as blobs
    return like + prior, (like_vect, np.random.normal((mu + tau * eta), sigma))


sampler_blobs = emcee.EnsembleSampler(
    nwalkers,
    ndim,
    lnprob_8school_blobs,
    args=(y_obs, sigma),
)
sampler_blobs.run_mcmc(pos, draws)

dims = {"eta": ["school"], "log_likelihood": ["school"], "y": ["school"]}
idata4 = az.from_emcee(
    sampler_blobs,
    var_names=["mu", "tau", "eta"],
    slices=[0, 1, slice(2, None)],
    arg_names=["y", "sigma"],
    arg_groups=["observed_data", "constant_data"],
    blob_names=["y", "y"],
    blob_groups=["log_likelihood", "posterior_predictive"],
    dims=dims,
    coords={"school": range(8)},
)
idata4

DataTree('None', parent=None)
├── DataTree('posterior')
│       Dimensions:  (draw: 1500, chain: 40, school: 8)
│       Coordinates:
│         * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
│         * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
│         * school   (school) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           mu       (draw, chain) float64 0.04892 0.4058 -0.4955 ... 0.92 6.357 6.915
│           tau      (draw, chain) float64 2.306 0.1492 2.226 ... 24.55 15.16 0.8366
│           eta      (draw, chain, school) float64 -0.6786 -0.1647 ... 0.1229 -0.3025
│       Attributes:
│           created_at:                 2023-06-16T00:40:47.329101
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('observed_data')
│       Dimensions:  (school: 8)
│       Coordinates:
│         * school   (school) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           y        (school) float64 28.0 8.0 -3.0 7.0 -1.0 1.0 18.0 12.0
│       Attributes:
│           created_at:                 2023-06-16T00:40:47.327326
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('constant_data')
│       Dimensions:      (sigma_dim_0: 8)
│       Coordinates:
│         * sigma_dim_0  (sigma_dim_0) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           sigma        (sigma_dim_0) float64 15.0 10.0 16.0 11.0 9.0 11.0 10.0 18.0
│       Attributes:
│           created_at:                 2023-06-16T00:40:47.327868
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('log_likelihood')
│       Dimensions:  (draw: 1500, chain: 40, school: 8)
│       Coordinates:
│         * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
│         * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
│         * school   (school) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           y        (draw, chain, school) float64 -3.872 -0.694 ... -1.206 -0.08795
│       Attributes:
│           created_at:                 2023-06-16T00:40:47.325055
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
├── DataTree('posterior_predictive')
│       Dimensions:  (draw: 1500, chain: 40, school: 8)
│       Coordinates:
│         * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
│         * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
│         * school   (school) int64 0 1 2 3 4 5 6 7
│       Data variables:
│           y        (draw, chain, school) float64 3.644 0.9363 -27.95 ... 8.866 -9.154
│       Attributes:
│           created_at:                 2023-06-16T00:40:47.326020
│           creation_library:           ArviZ
│           creation_library_version:   0.1
│           creation_library_language:  Python
│           inference_library:          emcee
│           inference_library_version:  3.1.4
└── DataTree('sample_stats')
        Dimensions:  (draw: 1500, chain: 40)
        Coordinates:
          * draw     (draw) int64 0 1 2 3 4 5 6 7 ... 1493 1494 1495 1496 1497 1498 1499
          * chain    (chain) int64 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39
        Data variables:
            lp       (draw, chain) float64 -23.76 -18.08 -24.63 ... -11.52 -

This last version, which contains both observed data and posterior predictive can be used to plot posterior predictive checks:

In [14]:
#az.plot_ppc(idata4, var_names=["y"], alpha=0.3, num_pp_samples=200);

In [15]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Fri Jun 16 2023

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 8.14.0

arviz_base: 0.1
numpy     : 1.24.3
emcee     : 3.1.4

Watermark: 2.3.1

